# Face Recognition

## Installing Required Packages
Important Note: during installing deepface package, in its dependancy, it downloads tensorflow cpu optimized package, and not the GPU version.

To Make deepface work on GPU: Create an environment with installed tensorlow-GPU Cuda support, then install the deepface package.

In [1]:
# !pip install deepface
# !pip install Deprecated
# !pip uninstall opencv-python -y
# !pip install opencv-contrib-python

## Importing Required Libararies and Packages

In [11]:
import warnings
warnings.filterwarnings('ignore')

import os
import time

import numpy as np, pandas as pd

import cv2

# from deepface import DeepFace

from deepface_modified import DeepFace

2023-09-18 19:03:38.471436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 19:03:39.093276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
os.getcwd()

'/home/yousri/Workspace/omdena/Orphanage/task_3_face_recognition/nb_workspace'

# Function Definitions

In [11]:
"""
    Modifiying the predefined method of the app class in InsightFace:FaceAnalysis Class, to have a label paramters to set on the image detection.
"""

def custom_draw_on(img, faces, label_lst):
    """
        Function used to draw rectangular box around the detected faces with their labels.
    """
    
    for i,face in enumerate(faces):
        box = face['facial_area']
        color = (0, 0, 255)
        cv2.rectangle(img, (box['x'], box['y']), (box['x']+box['w'], box['y']+box['h']), color, 2)
        cv2.putText(img, label_lst[i], (box['x']-1, box['y']-4),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)

    return img

def load_label_csv(label_path)->pd.DataFrame:
    """
        Function used to load csv label file
    """
    # Load Label CSV file
    # Step: Searching for a csv file locatted in this directory
    for file in os.listdir(label_path):
        if file.endswith(".csv"): 
            csv_path = os.path.join(label_path, file)

    # Step: load the csv label file
    return pd.read_csv(csv_path)


def face_recognition(
        img_path=None,
        label_path=None,
        face_detection_model='retinaface',
        face_recognition_model='ArcFace',
        enforce_detection=True,
        ):
    """
        This function could be used using the original deepface package without any modification in the package.
    """
    # Load Label CSV file
    df_label = load_label_csv(label_path=label_path)

    # Detect Faces
    detected_faces = DeepFace.extract_faces(
        img_path=img_path,
        detector_backend=face_detection_model
    )

    # Recognitize Faces with labels in database
    found_faces = DeepFace.find(img_path=img_path,
                db_path=label_path,
                model_name=face_recognition_model,
                detector_backend=face_detection_model,
                enforce_detection=enforce_detection)

    # Define label list
    label_lst = []

    for found_face in found_faces:
        
        img_label = 'UnKown'

        if len(found_face) != 0:

            # Face was detected
            # Pick the highest similarity
            found_face = found_face[
                found_face.ArcFace_cosine == found_face.ArcFace_cosine.max()
                ]
            
            # checking the ArcFace_cosine threshold
            thr = 0.25
            
            if found_face['ArcFace_cosine'].values[0]> thr:
                img_label_path = found_face['identity'].values[0]
                img_label = img_label_path.split('/')[-1].split('.')[0]
                # img_label = df_label[df_label.image_name == img_label_path[img_label_path.rfind('/') + 1:]]['label'].values[0]
                
        label_lst.append(img_label)

        
    if isinstance(img_path, str):
        img = cv2.imread(img_path)

        rimg= custom_draw_on(img, detected_faces, label_lst)

        print(f"Recognitioned Faces in {img_path} are: {label_lst}. Exported image with detected faces at: ../report/plots/{img_path.split('/')[-1]}")

    else:
        rimg= custom_draw_on(img_path, detected_faces, label_lst)

    return rimg


def face_recognition_optimized(
        img_path=None,
        label_path=None,
        face_detection_model='retinaface',
        face_recognition_model='ArcFace',
        enforce_detection=True,
        ):
    """
        This function could only be used by using the modified deepface package.
    """
    # Load Label CSV file
    df_label = load_label_csv(label_path=label_path)

    # Detect Faces
    detected_faces = DeepFace.extract_faces(
        img_path=img_path,
        detector_backend=face_detection_model
    )

    found_faces = DeepFace.find_modified(detected_faces,
                db_path=label_path,
                model_name=face_recognition_model,
                detector_backend='skip',
                enforce_detection=enforce_detection)

    # Define label list
    label_lst = []

    for found_face in found_faces:
        
        img_label = 'UnKown'

        if len(found_face) != 0:

            # Face was detected
            # Pick the highest similarity
            found_face = found_face[
                found_face.ArcFace_cosine == found_face.ArcFace_cosine.max()
                ]
            
            # checking the ArcFace_cosine threshold
            thr = 0.25
            
            if found_face['ArcFace_cosine'].values[0]> thr:
                img_label_path = found_face['identity'].values[0]
                img_label = img_label_path.split('/')[-1].split('.')[0]
                # img_label = df_label[df_label.image_name == img_label_path[img_label_path.rfind('/') + 1:]]['label'].values[0]
                
        label_lst.append(img_label)

    if isinstance(img_path, str):
        img = cv2.imread(img_path)

        rimg= custom_draw_on(img, detected_faces, label_lst)

        print(f"Recognitioned Faces in {img_path} are: {label_lst}. Exported image with detected faces at: ../report/plots/{img_path.split('/')[-1]}")

    else:
        rimg= custom_draw_on(img_path, detected_faces, label_lst)

    return rimg

## Define Model Parameters

In [12]:
face_recognition_model = 'ArcFace'
face_detection_model = 'retinaface'

# To force use GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Single Image Recognition
Importing Note: In case of adding new labels into the label dataset, you have to delete the representation.pkl file.

In [13]:
test_images_directory = '../data/test/'
label_path = '../data/label'
test_img = '003_57612506.jpg'

rimg = face_recognition(
    img_path=test_images_directory+test_img,
    label_path=label_path,
    face_detection_model=face_detection_model,
    face_recognition_model=face_recognition_model
)

# Output the labeled Image
cv2.imwrite(f"../report/plots/{test_img}", rimg)

There are  13  representations found in  representations_arcface.pkl
find function lasts  1.9721884727478027  seconds
Recognitioned Faces in ../data/test/003_57612506.jpg are: ['Angelina Jolie']. Exported image with detected faces at: ../report/plots/003_57612506.jpg


True

## Optimizing the face recognition function

In [8]:
test_images_directory = '../data/test/'
label_path = '../data/label'
test_img = '003_57612506.jpg'

rimg = face_recognition_optimized(
    img_path=test_images_directory+test_img,
    label_path=label_path,
    face_detection_model=face_detection_model,
    face_recognition_model=face_recognition_model
)

# Output the labeled Image
cv2.imwrite(f"../report/plots/{test_img}", rimg)

There are  13  representations found in  representations_arcface.pkl
find function lasts  0.1484386920928955  seconds
Recognitioned Faces in ../data/test/003_57612506.jpg are: ['UnKown']. Exported image with detected faces at: ../report/plots/003_57612506.jpg


True

Experimental Results are that the function excution time decreases.

# Video Image

In [10]:
test_images_directory = '../data/test/'
label_path = '../data/label'

# Create a video capture object
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture(f"{test_images_directory}Video3_chunk_8.mp4")

# Check if the video is opened successfully
if not cap.isOpened():
    print('Could not open the video file')
    exit()

# Set the capture buffer size
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Set the desired FPS
# cap.set(cv2.CAP_PROP_FPS, 60)

# Loop through the video frames
while True:
    
    ret, frame = cap.read()
    
    frame = face_recognition_optimized(
        img_path=frame,
        label_path=label_path,
        face_detection_model=face_detection_model,
        face_recognition_model=face_recognition_model
    )
    
    cv2.imshow('frame', frame)

    # Exit the loop if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
 
cv2.destroyAllWindows()

There are  13  representations found in  representations_arcface.pkl
find function lasts  0.13845562934875488  seconds
There are  13  representations found in  representations_arcface.pkl
find function lasts  0.14006543159484863  seconds
There are  13  representations found in  representations_arcface.pkl
find function lasts  0.14933514595031738  seconds
There are  13  representations found in  representations_arcface.pkl
find function lasts  0.16929340362548828  seconds
There are  13  representations found in  representations_arcface.pkl
find function lasts  0.14266037940979004  seconds
There are  13  representations found in  representations_arcface.pkl
find function lasts  0.1362602710723877  seconds
There are  13  representations found in  representations_arcface.pkl
find function lasts  0.14474725723266602  seconds
There are  13  representations found in  representations_arcface.pkl
find function lasts  0.14725732803344727  seconds
There are  13  representations found in  represen